In [5]:
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns 
from custom_transforms.transforms import *
from statsmodels.tsa.stattools import adfuller
from scripts.load_data import load_weather_df, load_price_df
from datetime import timedelta

In [6]:
#Load and merge data

price_weather_weekly_df = pd.read_csv('../processed_data/price_weather_weekly_df.V2.csv', index_col=0, parse_dates=True)
price_weather_weekly_df.index = pd.to_datetime(price_weather_weekly_df.index)
price_weather_weekly_df

,Alface Americana - Roça,Alface Crespa - Roça,Alface Lisa - Roça,Alface Americana - Atacado,Alface Crespa - Atacado,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)_mean","PRECIPITAÇÃO TOTAL, HORÁRIO (mm)_max","PRECIPITAÇÃO TOTAL, HORÁRIO (mm)_sum","PRECIPITAÇÃO TOTAL, HORÁRIO (mm)_min","PRECIPITAÇÃO TOTAL, HORÁRIO (mm)_std",...,"VENTO, RAJADA MAXIMA (m/s)_median","VENTO, RAJADA MAXIMA (m/s)_ma4","VENTO, VELOCIDADE HORARIA (m/s)_mean","VENTO, VELOCIDADE HORARIA (m/s)_max","VENTO, VELOCIDADE HORARIA (m/s)_sum","VENTO, VELOCIDADE HORARIA (m/s)_min","VENTO, VELOCIDADE HORARIA (m/s)_std","VENTO, VELOCIDADE HORARIA (m/s)_var","VENTO, VELOCIDADE HORARIA (m/s)_median","VENTO, VELOCIDADE HORARIA (m/s)_ma4"
dt,,,,,,,,,,,,,,,,,,,,,
2017-04-30 00:00:00+00:00,0.849167,0.513889,0.550000,NaN,NaN,0.391667,12.0,65.8,0.0,1.130387,...,3.00,1.275,0.600595,3.6,100.9,0.0,0.816337,0.666407,0.1,0.075
2017-05-07 00:00:00+00:00,0.781667,0.541667,0.513889,NaN,NaN,0.070238,8.4,11.8,0.0,0.666078,...,0.00,0.000,0.141071,2.5,23.7,0.0,0.380880,0.145070,0.0,0.000
2017-05-14 00:00:00+00:00,0.725000,0.467778,0.444444,NaN,NaN,0.108333,4.6,18.2,0.0,0.495176,...,2.35,1.650,0.341071,3.0,57.3,0.0,0.698701,0.488183,0.0,0.175
2017-05-21 00:00:00+00:00,0.698333,0.375000,0.437778,NaN,NaN,0.302381,7.2,50.8,0.0,1.065291,...,3.05,2.425,0.552976,3.4,92.9,0.0,0.819517,0.671608,0.0,0.175
2017-05-28 00:00:00+00:00,0.666667,0.322222,0.322222,NaN,NaN,0.153571,7.4,25.8,0.0,0.829970,...,3.70,0.000,0.721429,3.1,121.2,0.0,0.872270,0.760855,0.2,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-03 00:00:00+00:00,1.875000,0.750000,0.750000,1.0,0.5,0.077381,5.2,13.0,0.0,0.490730,...,3.80,2.225,1.291071,4.3,216.9,0.1,0.784476,0.615402,1.2,0.600
2023-12-10 00:00:00+00:00,1.458333,0.677083,0.677083,1.0,0.5,0.697619,28.2,117.2,0.0,2.583595,...,3.10,4.075,0.982738,2.0,165.1,0.2,0.439787,0.193413,0.9,0.950
2023-12-17 00:00:00+00:00,1.125000,0.562500,0.562500,1.0,0.5,0.347619,28.2,58.4,0.0,2.252901,...,3.40,2.325,1.075595,2.7,180.7,0.2,0.531465,0.282455,1.0,0.775


In [7]:
def safe_filename(filename):
    keepcharacters = (' ', '.', '_')
    return "".join(c for c in filename if c.isalnum() or c in keepcharacters).rstrip()


In [8]:
interest_cols = ['Alface Crespa - Roça']
shift_cols = price_weather_weekly_df.columns.tolist()
#shift_cols = ['RADIACAO GLOBAL (Kj/m²)_max']

with open('2024-05-12-individual-correlation/correlation.csv', 'w') as f:
    f.write('column,correlated_to,max_correlation\n')

for col in shift_cols:
    temp_df = pd.DataFrame()
    temp_df.index = price_weather_weekly_df.index
   
    for i_col in interest_cols:
        temp_df[i_col] = price_weather_weekly_df[i_col]
    
    temp_df[0] = price_weather_weekly_df[col]
    for i in range(1, 110):
        temp_df[i] = temp_df[0].shift(i)
        temp_df = temp_df.copy()
    
    corr = temp_df.corr(method="spearman")
    corr = corr.iloc[1:]
    corr = corr[interest_cols]
    corr = corr[interest_cols].apply(lambda x: x.abs())
    max_corr = corr[interest_cols].max().values[0]
    corr.index.names = [f'Shifted: {col}']
    corr.columns = [f'Corr. with {c}' for c in interest_cols]
    

    plt.tight_layout()
    
    corr.plot(figsize=(10, 3), ylim=(0, 1), title=f'Correlation between {col} and {interest_cols[0]}', grid=True)
    
    vertical_lines_1 = [i for i in range(0, 110, 26)]
    for v in vertical_lines_1:
        plt.axvline(x = v, color = 'r', label = 'axvline - full height')
        plt.axvline(x = v+7, color = 'b', label = 'axvline - full height')
        plt.axvline(x = v+10, color = 'g', label = 'axvline - full height')
        
    with open('2024-05-12-individual-correlation/correlation.csv', 'a') as f:
        f.write(f'\"{col}\",\"{interest_cols[0]}\",{max_corr}\n')
    
    plt.savefig(f'2024-05-12-individual-correlation/{safe_filename(col)}.png', bbox_inches='tight')
    plt.close()



<Figure size 640x480 with 0 Axes>